### A Simple Function

In [ ]:
import json

In [28]:
def get_current_weather(location: str, unit: str):
    """

    """
    if location == "Hà Nội":
        return json.dumps({"Temperature": 20, "unit": unit})
    else:
        return json.dumps({"Temperature": 50, "unit": unit})

In [29]:
get_current_weather("Hà Nội", unit="Độ C")

'{"Temperature": 20, "unit": "\\u0110\\u1ed9 C"}'

In [36]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
Bạn là một mô hình AI gọi hàm. Bạn được cung cấp các chữ ký hàm trong thẻ XML <tools></tools>.
Bạn có thể gọi một hoặc nhiều hàm để hỗ trợ truy vấn của người dùng. Đừng đưa ra giả định về giá trị cần truyền vào hàm.
Hãy đặc biệt chú ý đến thuộc tính 'types'. Bạn nên sử dụng các kiểu dữ liệu này giống như trong một từ điển Python.
Đối với mỗi lần gọi hàm, hãy trả về một đối tượng JSON với tên hàm và các đối số trong thẻ XML <tool_call></tool_call> như sau:
<tool_call>

{"name": <tên hàm>,"arguments": <từ điển đối số>}

</tool_call>

Đây là các công cụ có sẵn:

<tools> {
    "name": "get_current_weather",
    "description": "Lấy thời tiết hiện tại tại một vị trí nhất định location (str): Thành phố và tiểu bang, ví dụ: Hà Nội, Danang unit (str): Đơn vị. Nó có thể nhận hai giá trị; 'độ C', 'độ F',
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [37]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "Nhiệt độ hiện tại ở Hà Nội theo độ C là bao nhiêu?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)


<tool_call>
{
    "name": "get_current_weather",
    "arguments": {
        "location": "Hà Nội",
        "unit": "độ C"
    }
}
</tool_call>


In [38]:
def parse_tool_call_str(tool_call_str: str):
    pattern = f'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output."

In [39]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Hà Nội', 'unit': 'độ C'}}

In [40]:
result = get_current_weather(**parsed_output['arguments'])  # ** unpack keyword arguments (giải nén tham số dạng key=value).

In [41]:
result

'{"Temperature": 20, "unit": "\\u0111\\u1ed9 C"}'

In [42]:
agent_chat_history.append(
    {
        "role": "user",
        "content": f"Observation: {result}"
    }
)

In [43]:
GROQ_CLIENT.chat.completions.create(
    messages = agent_chat_history,
    model = MODEL
).choices[0].message.content

'Nhiệt độ hiện tại ở Hà Nội là 20 độ C.'

In [6]:
%cd ..

/home/ngocson/NSN/Project/ai-agents-from-scratch


In [44]:
import json
import requests
from src.agentic_patterns.tool_pattern.tool import *
from src.agentic_patterns.tool_pattern.tool_agent import ToolAgent

In [45]:
import json
import requests
from src.agentic_patterns.tool_pattern.tool import tool
from src.agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. 
    Each story contains the title, URL, score, author, and time of submission. The data is fetched 
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    
    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Get the top story IDs
        top_story_ids = response.json()[:top_n]
        
        top_stories = []
        
        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()
            
            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })
        
        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return [] 

In [46]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'A 26,000-year astronomical monument hidden in plain sight (2019)',
  'url': 'https://longnow.org/ideas/the-26000-year-astronomical-monument-hidden-in-plain-sight/'},
 {'title': 'Are arrays functions?',
  'url': 'https://futhark-lang.org/blog/2026-01-16-are-arrays-functions.html'},
 {'title': 'California is free of drought for the first time in 25 years',
  'url': 'https://www.latimes.com/california/story/2026-01-09/california-has-no-areas-of-dryness-first-time-in-25-years'},
 {'title': "Claude Chill: Fix Claude Code's Flickering in Terminal",
  'url': 'https://github.com/davidbeesley/claude-chill'},
 {'title': 'Instabridge has acquired Nova Launcher',
  'url': 'https://novalauncher.com/nova-is-here-to-stay'}]

In [47]:
hn_tool = tool(fetch_top_hacker_news_stories)

In [48]:
tool_agent = ToolAgent(tools=[hn_tool])

In [52]:
output=tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now.")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'type': 'function', 'name': 'fetch_top_hacker_news_stories', 'parameters': {'top_n': '5'}, 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "A 26,000-year astronomical monument hidden in plain sight (2019)", "url": "https://longnow.org/ideas/the-26000-year-astronomical-monument-hidden-in-plain-sight/"}, {"title": "Are arrays functions?", "url": "https://futhark-lang.org/blog/2026-01-16-are-arrays-functions.html"}, {"title": "California is free of drought for the first time in 25 years", "url": "https://www.latimes.com/california/story/2026-01-09/california-has-no-areas-of-dryness-first-time-in-25-years"}, {"title": "Claude Chill: Fix Claude Code's Flickering in Terminal", "url": "https://github.com/davidbeesley/claude-chill"}, {"title": "Instabridge has acquired Nova Launcher", "url": "https://novalauncher.com/nova-is-here-to-stay"}]


In [53]:
print(output)

Based on the provided data, here are the top 5 Hacker News stories:

1. **A 26,000-year astronomical monument hidden in plain sight (2019)** - https://longnow.org/ideas/the-26000-year-astronomical-monument-hidden-in-plain-sight/
2. **Are arrays functions?** - https://futhark-lang.org/blog/2026-01-16-are-arrays-functions.html
3. **California is free of drought for the first time in 25 years** - https://www.latimes.com/california/story/2026-01-09/california-has-no-areas-of-dryness-first-time-in-25-years
4. **Claude Chill: Fix Claude Code's Flickering in Terminal** - https://github.com/davidbeesley/claude-chill
5. **Instabridge has acquired Nova Launcher** - https://novalauncher.com/nova-is-here-to-stay

Please note that the ranking might not reflect the real-time ranking on Hacker News, as the data provided is a snapshot and may not be up-to-date.
